In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import differential_evolution

In [2]:
#read data
d = pd.read_csv('/home/petacapek/Dokumenty/pracovni/data_statistika/Data_z_clanku/Marstorp1999/Marstorp1999.csv',
               sep='\t')

#d = d[(d.time<0.3) | (d.time>0.8)]

print(d)
d.shape

                        Study        Soil Substrate  Clay   pH  Ctot  Ntot  \
0   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
1   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
2   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
3   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
4   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
5   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
6   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
7   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
8   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
9   Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
10  Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1   1.7   NaN   
11  Marstorp and Witter, 1999  Sandy loam   Glucose    10  6.1  

(41, 18)

# Two pool model is defined

In [3]:
def Twopool (y, t, pars):
    #define initial states
    G=y[0];    Br=y[1];    Bs=y[2]
        
    #define parameters
    v=pars[0];   k=pars[1];     f=pars[2] 
    m=pars[3];   Y=pars[4]
                        
    #Define fluxes
    ##glucose uptake
    uptake=v*Bs*G/(k+G)
    ##transfer function
    transfer=f*Br
    #death rate
    death=m*Bs
            
    #Define derivatives
    dGdt=-uptake
    dBrdt=uptake-transfer
    dBsdt=transfer*Y-death
    
    return dGdt, dBrdt, dBsdt;

## Additional calculations are performed

In [17]:
def calc (model, pars, t, y0):
    #model parameters
    pars_model=pars[0:5]
    #conversion factors
    conversions=pars[5:7]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
       
    #calculate respiration rates
    r=pars_model[2]*y[:, 1]*(1-pars_model[4])
    
    #calculate CFC
    fs=d.Cmicinit[0]*conversions[1]/d.DNAinit[0]
    CFC = conversions[0]*y[:, 1]+fs*y[:, 2]
    CFC14 = (conversions[0]*y[:, 1]+fs*y[:, 2])-d.Cmicinit[0]
    
    #calculate DNA
    DNA = conversions[1]*y[:, 2]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           r.reshape(len(d.Time),1),
                           #CFC.reshape(len(d.Time),1),
                           CFC14.reshape(len(d.Time),1),
                           DNA.reshape(len(d.Time),1)), axis=1)
    
    return yhat

## Objective function is defined

In [18]:
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    G_i = d.Sinit[0]
    Bs_i = d.DNAinit[0]/pars[6]
    #Br_i = (d.Cmicinit[0]-Bs_i*pars[6])/pars[5]      
    y0 = np.array([G_i, 0, Bs_i])
    
    #times
    t = d.Time
    
    #model simulations
    yhat_full = calc(Twopool, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO212]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14+d.Cmic12]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO212).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic14+d.Cmic12)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.DNA/5).repeat(len(d.Time)).reshape(len(d.Time),1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

## Goodness of fit

In [19]:
def goodness (x):
    #define parameters
    pars = x
    
    #initial conditions
    G_i = d.Sinit[0]
    Bs_i = d.DNAinit[0]/pars[6]
    #Br_i = (d.Cmicinit[0]-Bs_i*pars[6])/pars[5]      
    y0 = np.array([G_i, 0, Bs_i])
    
    #times
    t = d.Time
    
    #model simulations
    yhat_full = calc(Twopool, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO212]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14+d.Cmic12]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)), 
                     axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    out = np.array([R2, ll, AIC])
          
    return out

## Parameters estimation

In [22]:
pars_opt=differential_evolution(obj_fun, [(1e-3, 10), (1e-3, 10), 
                                          (1e-3, 10), (1e-8, 1),
                                          (0, 1), (0, 1),
                                          (0, 1)], 
                                  polish=True, maxiter=1000000)

print(pars_opt)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
/home/petacapek/.local/lib/python3.6/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in multiply


     fun: 11.722808572858648
     jac: array([ 1.18273924e-01,  1.26414736e+00,  1.18594912e-02,  2.30845387e-01,
        1.54977542e-01, -1.67865721e-04, -3.87491994e-01])
 message: 'Optimization terminated successfully.'
    nfev: 10428
     nit: 91
 success: True
       x: array([5.96960936e-01, 1.00000000e-03, 3.06296385e+00, 2.18985875e-02,
       9.89421781e-01, 2.69237782e-01, 1.54466937e-02])


In [23]:
print(goodness(pars_opt.x))
np.savetxt('opt_pars.csv', pars_opt.x.reshape(1,7), delimiter=",")

[ 0.97480358 -0.8692765  15.73855301]


# Monod model is defined

In [199]:
def Monod (y, t, pars):
    #define initial states
    G=y[0];    B=y[1]
        
    #define parameters
    v=pars[0];   k=pars[1];
    m=pars[2];   Y=pars[3]
                        
    #Define fluxes
    ##glucose uptake
    uptake=v*B*G/(k+G)
    ##decay
    decay=B*m
            
    #Define derivatives
    dGdt=-uptake
    dBdt=Y*uptake-decay
       
    
    return dGdt, dBdt;

## Additional calculations are performed

In [200]:
def calc (model, pars, t, y0):
    #model parameters
    pars_model=pars[0:4]
    #conversion factors
    conversions=pars[4:6]
    
    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))
       
    #calculate respiration rates
    r=pars_model[0]*y[:, 0]*y[:, 1]/(pars_model[1]+y[:, 0])*(1-pars_model[3])+(pars_model[2]*y[:, 1])
    
    #calculate CFC
    CFC = conversions[0]*y[:, 1]
    CFC14 = (conversions[0]*y[:, 1])-d.Cmicinit[0]
    #calculate DNA
    DNA = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           r.reshape(len(d.Time),1),
                           #CFC.reshape(len(d.Time),1),
                           CFC14.reshape(len(d.Time),1),
                           DNA.reshape(len(d.Time),1)), axis=1)
    
    return yhat

## Objective function is defined

In [201]:
def obj_fun (x):
    #define parameters
    pars = x
    
    #initial conditions
    G_i = d.Sinit[0]
    B_i = d.Cmicinit[0]/pars[4]
    y0 = np.array([G_i, B_i])
    
    #times
    t = d.Time
    
    #model simulations
    yhat_full = calc(Monod, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO212]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14+d.Cmic12]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)), 
                     axis=1)
    
    #weights
    weights=np.concatenate((np.nanmean(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.CO212).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean((d.Cmic14+d.Cmic12)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d.DNA/5).repeat(len(d.Time)).reshape(len(d.Time),1)), 
                       axis=1)
                
          
    out=np.nansum(((yhat_full-obs)/weights)**2)
          
    return out

## Goodness of fit

In [202]:
def goodness (x):
    #define parameters
    pars = x
    
    #initial conditions
    G_i = d.Sinit[0]
    B_i = d.Cmicinit[0]/pars[4]
    y0 = np.array([G_i, B_i])
    
    #times
    t = d.Time
    
    #model simulations
    yhat_full = calc(Monod, pars, t, y0)
         
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        np.array([d.CO212]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14+d.Cmic12]).reshape(len(d.Time),1),
                        np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.DNA]).reshape(len(d.Time),1)), 
                     axis=1)
    
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    out = np.array([R2, ll, AIC])
          
    return out

## Parameters estimation

In [203]:
monod_opt=differential_evolution(obj_fun, [(1e-5, 10), (1e-5, 100),
                                          (1e-8, 1), (0, 1), 
                                          (0, 1), (0, 0.1)], 
                                  polish=True, maxiter=1000000)

print(monod_opt)

     fun: 17.196849317801696
 message: 'Optimization terminated successfully.'
    nfev: 14061
     nit: 155
 success: True
       x: array([6.77383349e-01, 1.52867907e-02, 7.75178762e-04, 9.92333921e-01,
       1.16884682e-01, 1.78518950e-02])


/usr/lib/python3/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/home/petacapek/.local/lib/python3.6/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in multiply
  # This is added back by InteractiveShellApp.init_path()
/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in multiply
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in multiply
  from ipykernel import kernelapp as app
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: inva

In [204]:
print(goodness(monod_opt.x))
np.savetxt('monod_opt.csv', monod_opt.x.reshape(1,6), delimiter=",")

[ 0.98005895 -0.68796624 13.37593248]
